# Analise de Emoções em Brinquedos de Parques de Diversões (IA)

O objetivo é determinar as emoções que são sentidas durante o uso dos brinquedos no parque de diversão, gerando um relatório com as emoções ponderadas de acordo com o áudio capturado. Assim podendo comparar as emoções que são esperadas para determinados brinquedos e as que realmente são obtidas. As emoções analisadas são: 

1. Felicidade
2. Medo
3. Desgosto
4. Tristeza
5. Neutra
6. Raiva 

E os brinquedos utilizados utilizados como exemplo no projeto, são:

1. Montanha Russa
2. Casa dos Monstros
3. Carrinho Bate-Bate
4. Roda Gigante


Importando as bibliotecas necessários para o projeto 

In [ ]:
import librosa
import librosa.display
import pandas as pd
import numpy as np
import os
from pydub import AudioSegment
from sklearn.tree import DecisionTreeClassifier as tree
from sklearn.model_selection import cross_val_score
from loop_listen.loop_listen import Loop_listen as Loop
import sklearn.tree as SKTree
import matplotlib.pyplot as plt
from reportlab.pdfgen import canvas

Declarando as variáveis globais. 

directories: Lista dos diretorios no qual estão localizados os audios para treinamento da IA (Inteligência Artificial) 

emotionStorage: Lista de arrays responsável pelo armazenamento obtidos durante a utilização dos brinquedos (gravação dos audios)

In [2]:
directories = ['/0_happy', '/1_fear', '/2_disgust', '/3_sad', '/4_neutral', '/5_angry']
emotionStorage = [[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0]]

Função responsável pela criação e organização dos dataset (base de dados) que serão utilizados. 

O dataset será composto pelas seguintes features (atributos): 

genre - Podendo ser 0 para mulher e 1 para homem.

mfcc - Técnica para extração de atributos com o objetivo de converter o espectro da voz para a escala MEL, uma escala que visa copiar as características básicas e únicas que o ouvido humano pode ouvir.

stft - É uma sequência de transformadas de Fourier, a qual fornece informações de frequências localizadas no tempo.

chromagram - Entende-se como um descrito que representa o material tonal de um sinal de áudio.

mel - Escala que visa copiar as características básicas e únicas que o ouvido humano pode ouvir.

contrast - Técnica utilizada para reduções de ruídos.
 
tonnetz - É uma representação plana entre os tons musicais.

emotion :  0: happy, 1: fear, 2: disgust, 3: sad, 4: neutral, 5: angry


In [3]:
def organizeDataFrames():
    aux = 0
    for dir in directories:
        print(dir)
        maleFiles = os.listdir('data/MaleSounds' + dir)
        df_male = pd.DataFrame(maleFiles)
        df_male['genre'] = '1'
        df_male['mfcc'] = '0'
        df_male['stft'] = '0'
        df_male['chromagram'] = '0'
        df_male['mel'] = '0'
        df_male['contrast'] = '0'
        df_male['tonnetz'] = '0'
        df_male['emotion'] = str(aux)
        df_male = df_male.rename(columns={0: 'file'})

        femaleFiles = os.listdir('data/FemaleSounds' + dir)
        df_female = pd.DataFrame(femaleFiles)
        df_female['genre'] = '0'
        df_female['mfcc'] = '0'
        df_female['stft'] = '0'
        df_female['chromagram'] = '0'
        df_female['mel'] = '0'
        df_female['contrast'] = '0'
        df_female['tonnetz'] = '0'
        df_female['emotion'] = str(aux)
        df_female = df_female.rename(columns={0: 'file'})

        if aux == 0:
            df1 = df_male
            df2 = df_female
            aux = aux + 1
        elif aux > 0:
            df1 = pd.concat([df1, df_male], ignore_index=True)
            df2 = pd.concat([df2, df_female], ignore_index=True)
            aux = aux + 1

    df_voices = pd.concat([df1, df2], ignore_index=True)

    return df_voices, df1, df2

In [4]:
Função responsável pelo processamento dos sinas dos audios da base de dados escolhida. Após isso, é armazenado no dataset.

SyntaxError: invalid syntax (<ipython-input-4-f2ddcefd46c6>, line 1)

In [5]:
def generateFeatures(df_voices, path_1, path_2):
    for x in df_voices.values:
        if x[1] == '1':
            if x[8] == '0':
                fileName = './data/MaleSounds/0_happy' + '/' + str(x[0])
            if x[8] == '1':
                fileName = './data/MaleSounds/1_fear' + '/' + str(x[0])
            if x[8] == '2':
                fileName = './data/MaleSounds/2_disgust' + '/' + str(x[0])
            if x[8] == '3':
                fileName = './data/MaleSounds/3_sad' + '/' + str(x[0])
            if x[8] == '4':
                fileName = './data/MaleSounds/4_neutral' + '/' + str(x[0])
            if x[8] == '5':
                fileName = './data/MaleSounds/5_angry' + '/' + str(x[0])
        if x[1] == '0':
            if x[8] == '0':
                fileName = './data/FemaleSounds/0_happy' + '/' + str(x[0])
            if x[8] == '1':
                fileName = './data/FemaleSounds/1_fear' + '/' + str(x[0])
            if x[8] == '2':
                fileName = './data/FemaleSounds/2_disgust' + '/' + str(x[0])
            if x[8] == '3':
                fileName = './data/FemaleSounds/3_sad' + '/' + str(x[0])
            if x[8] == '4':
                fileName = './data/FemaleSounds/4_neutral' + '/' + str(x[0])
            if x[8] == '5':
                fileName = './data/FemaleSounds/5_angry' + '/' + str(x[0])
                
        sound, sr = librosa.load(fileName, res_type='kaiser_fast')
        # mel-frequency cepstral coefficient
        mfcc = np.mean(librosa.feature.mfcc(y=sound, sr=sr, n_mfcc=40).T, axis=0)
        x[2] = np.mean(mfcc)
        # Fourier transformation
        stft = np.abs(librosa.stft(sound))
        x[3] = np.mean(stft)
        # Chromagram
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sr).T, axis=0)
        x[4] = np.mean(chroma)
        # mel-scaled spectrogram
        mel = np.mean(librosa.feature.melspectrogram(sound, sr=sr).T, axis=0)
        x[5] = np.mean(mel)
        # spectral contrast
        contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sr).T, axis=0)
        x[6] = np.mean(contrast)
        # tonnetz
        tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(sound),
                                                  sr=sr).T, axis=0)
        x[7] = np.mean(tonnetz)

    df_target = df_voices.drop(['file', 'mfcc', 'stft', 
                                'chromagram', 'mel', 'contrast', 'tonnetz'], axis=1)
    
    df_voices = df_voices.drop(['file'], axis=1)

    df_voices.to_csv(path_2, index=False)
    df_target.to_csv(path_1, index=False)

    return df_voices, df_target

SplitAudio é utilizada para a divisão de um intevalo de audio 10 segundos em pedaços menores, de 1 segundo cada. 

#reference: https://stackoverflow.com/questions/37999150/how-to-split-a-wav-file-into-multiple-wav-files

In [6]:
def splitAudio(path):
    t1 = 0
    t2 = 1
    tt1 = 0
    tt2 = 0
    audioList = []
    while t2 <= 10:
        print(t1)
        tt1 = t1 * 1000  # Works in milliseconds
        tt2 = t2 * 1000

        newAudio = AudioSegment.from_wav(path)
        newAudio = newAudio[tt1:tt2]
        audioList.append(newAudio)
        newAudio.export(str(t1) + '.wav', format="wav")  # Exports to a wav file in the current path.
        t1 = t1 + 1
        t2 = t2 + 1

    return audioList

Função utilizada para o processamento do sinal do audio capturado e o transformando em um dataframe para ser utilizado posteriormente na predição

In [7]:
def generateAudioFeatures(path):
    fileName = path
    print(path)
    d = {'file': ['file'], 'genre': ['0'], 'mfcc': ['0'],'stft': ['0'],'chromagram': ['0'],'mel': ['0'],
         'contrast': ['0'],'tonnetz': ['0'],'emotion': ['0']}
    df = pd.DataFrame(d, columns=['file', 'genre', 'mfcc', 'stft', 'chromagram', 'mel', 'contrast', 'tonnetz', 'emotion'])

    for Z in df.values:
        sound, sr = librosa.load(fileName, res_type='kaiser_fast')
        mfcc = np.mean(librosa.feature.mfcc(y=sound, sr=sr, n_mfcc=40).T, axis=0)
        Z[2] = np.mean(mfcc)
        # Fourier transformation
        stft = np.abs(librosa.stft(sound))
        Z[3] = np.mean(stft)
        # Chromagram
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sr).T, axis=0)
        Z[4] = np.mean(chroma)
        # mel-scaled spectrogram
        mel = np.mean(librosa.feature.melspectrogram(sound, sr=sr).T, axis=0)
        Z[5] = np.mean(mel)
        # spectral contrast
        contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sr).T, axis=0)
        Z[6] = np.mean(contrast)
        # tonnetz
        tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(sound),
                                                  sr=sr).T, axis=0)
        Z[7] = np.mean(tonnetz)
    df = df.drop(['file'], axis=1)
    return df

Função responsável pela classificação do genero do usuário. Utilizando o classificador com a seguinte parametrização: 

- random_state=2020

- criterion='gini'

- max_depth=6

Os atributos utilizados foram: stft, chromagram, mel, contrast.

A validação do modelo se deu pelo método de validação cruzado (cross-validation), e 5 distribuições de dados diferentes. 
    

In [8]:
def genreClassify(df_voices, df_target, path):
    classifier_dt = tree(random_state=2020, criterion='gini', max_depth=6)
    classifier_dt.fit(df_voices[['stft', 'chromagram',
                      'mel', 'contrast']], df_target['genre'])

    scores_dt = cross_val_score(classifier_dt, df_voices[['stft', 'chromagram', 'mel', 'contrast']],
                                df_target['genre'], scoring='accuracy', cv=5)
    print("Acurácia de Gênero: " + str(scores_dt.mean()))
    #SKTree.plot_tree(classifier_dt, fontsize=10)
    #plt.show()

    df_answer = generateAudioFeatures(path + '.wav')

    x = float(classifier_dt.predict(df_answer[['stft', 'chromagram', 'mel', 'contrast']]))

    #for x in df_voices['stft', 'chromagram', 'mel', 'contrast']

    print("Resultado: " + str(x))
    if x == 0:
        print("Áudio de uma mulher")
        return 0, df_answer

    elif x == 1:
        print("Áudio de um homem")
        return 1, df_answer

    return -1, df_answer

manEmotionClassify e womanEmotionClassify são os classificadores de emoções, para isso foram empregados os seguintes parametros: 

Para homem: 

- random_state=2020
- criterion='entropy'
- max_depth=7

Para mulher: 

- random_state=2020
- criterion='entropy'
- max_depth=8


O retorno tem o resultado da emoção capturada, de acordo com a lista de emoções previamente informada. 

In [9]:
def manEmotionClassify(df_man_voices, df_man_target, df_predict):
    classifier_manEmotion = tree(random_state=2020, criterion='entropy', max_depth=7)
    classifier_manEmotion.fit(df_man_voices[['mfcc', 'stft', 'chromagram',
                              'mel', 'contrast', 'tonnetz']], df_man_target['emotion'])

    scores_dt = cross_val_score(classifier_manEmotion, df_man_voices[['mfcc', 'stft',
                                'chromagram', 'mel', 'contrast', 'tonnetz']],
                                df_man_target['emotion'], scoring='accuracy', cv=5)

    print("Acurácia de Emoção H: " + str(scores_dt.mean()))
    answer = classifier_manEmotion.predict(df_predict[['mfcc', 'stft', 'chromagram',
                              'mel', 'contrast', 'tonnetz']])
    print(answer)
    if answer[0] == 0:
        return answer[0]

    elif answer[0] == 1:
        return answer[0]

    elif answer[0] == 2:
        return answer[0]

    elif answer[0] == 3:
        return answer[0]

    elif answer[0] == 4:
        return answer[0]

    elif answer[0] == 5:
        return answer[0]

    return -1

In [10]:

def womanEmotionClassify(df_woman_voices, df_woman_target, df_predict):
    classifier_womanEmotion = tree(random_state=2020, criterion='entropy', max_depth=8)
    classifier_womanEmotion.fit(df_woman_voices[['mfcc', 'stft', 'chromagram',
                                'mel', 'contrast', 'tonnetz']], df_woman_target['emotion'])

    scores_dt = cross_val_score(classifier_womanEmotion, df_woman_voices[['mfcc', 'stft',
                                'chromagram', 'mel', 'contrast', 'tonnetz']],
                                df_woman_target['emotion'], scoring='accuracy', cv=5)

    print("Acurácia de Emoção M: " + str(scores_dt.mean()))

    answer = classifier_womanEmotion.predict(df_predict[['mfcc', 'stft', 'chromagram',
                              'mel', 'contrast', 'tonnetz']])
    print(answer)
    if answer[0] == 0:
        return answer[0]

    elif answer[0] == 1:
        return answer[0]

    elif answer[0] == 2:
        return answer[0]

    elif answer[0] == 3:
        return answer[0]

    elif answer[0] == 4:
        return answer[0]

    elif answer[0] == 5:
        return answer[0]

    return -1


Função para limpar o armazenamento na lista de arrays das emoções 

In [11]:
def resetValues():
    global emotionStorage

    emotionStorage = [[0, 0, 0, 0, 0, 0], 
                      [0, 0, 0, 0, 0, 0], 
                      [0, 0, 0, 0, 0, 0], 
                      [0, 0, 0, 0, 0, 0]]
    return

Função que gera o relatório utilizando a lista de brinquedos que estão disponíveis para serem analisados e os dados resultantes da analise feita pela IA. 

In [12]:
def buildReport(emotionsRecorded):

    rides = ['Roller Coaster', 'House of Monsters', 'Bumper Cars', 'Ferris Wheel']

    emotions = ['Happy', 'Fear', 'Disgust', 'Sad', 'Neutral', 'Angry']
    percentages = []

    c = canvas.Canvas("report.pdf")

    c.setFillColor('grey')
    c.setFont('Helvetica-Bold', 18)
    c.drawString(400, 800, 'Rides Report')

    count = 0
    countRides = 0

    for ride in rides:
        c.setFillColor('grey')
        c.setFont('Helvetica', 16)
        c.drawString(50,750 - (count * 30), ride)
        count += 1

        for xTimesRecognized in emotionsRecorded[countRides]:
            percentages.append(str((xTimesRecognized/10) * 100) + '%')

        countRides += 1

        for x in range(6):
            c.setFillColor('grey')
            c.setFont('Helvetica', 12)
            c.drawString(100,750 - (count * 30), emotions[x])

            c.setFillColor('red')
            c.setFont('Helvetica', 12)
            c.drawString(200,750 - (count * 30), percentages[x])
            count += 1

        count += 1
        percentages = []

        if countRides == 3:
            c.showPage()
            count = 0

    c.save()

Main - Carregando os arquivos e criando os datasets

In [13]:
aux = 0
allFiles = os.listdir('./')
for x in allFiles:
    if x == 'Target.csv':
        aux = aux + 1
    elif x == 'Voices.csv':
        aux = aux + 1
    elif x == 'Voices_Male.csv':
        aux = aux + 1
    elif x == 'Target_Male.csv':
        aux = aux + 1
    elif x == 'Voices_Female.csv':
        aux = aux + 1
    elif x == 'Target_Female.csv':
        aux = aux + 1

if aux < 6:
    print("Criando dataset")
    df, df_male, df_female = organizeDataFrames()
    df_voices, df_target = generateFeatures(df, r'Target.csv', r'Voices.csv')
    df_male_voices, df_male_target = generateFeatures(df_male, r'Target_Male.csv', r'Voices_Male.csv')
    df_female_voices, df_female_target = generateFeatures(df_female, r'Target_Female.csv', r'Voices_Female.csv')
    df_voices.head(600)
elif aux == 6:
    print("Recebendo dataset")
    df_voices = pd.read_csv('Voices.csv', sep=',', index_col=None)
    df_target = pd.read_csv('Target.csv', sep=',', index_col=None)
    df_male_voices = pd.read_csv('Voices_Male.csv', sep=',', index_col=None)
    df_male_target = pd.read_csv('Target_Male.csv', sep=',', index_col=None)
    df_female_voices = pd.read_csv('Voices_Female.csv', sep=',', index_col=None)
    df_female_target = pd.read_csv('Target_Female.csv', sep=',', index_col=None)


Recebendo dataset


Main - Mostrando o menu principal com as opções de escolha dos brinquedos ou opções de sistema

In [ ]:


###audio capture
option = -1
while 1:
    print("0- Roller Coaster\n1- House of Monsters\n2- Bumper Cars\n3- Ferris Wheel\
    n4- Display Data\n5- Generate a Report\n6- Reset Data\n7- Exit")
    
    option = int(input("Option: "))
    
    while option < 0 and option > 7:
        option = int(input("Invalid Option, insert your option again: "))
    audio = Loop(filename="audioTest", threshold=False, max_seconds=10)
    if option != 4 and option != 5 and option != 6 and option != 7:
        print("Ouvindo")
        audio.listen()
        audioList = splitAudio('./output/audio/audioTest.wav')
        x = 0
        for i in audioList:
            answer, df_answer = genreClassify(df_voices, df_target, str(x))
            x = x + 1

            if answer == 0:
                emotionAnswer = womanEmotionClassify(df_female_voices, df_female_target, df_answer)

            elif answer == 1:
                emotionAnswer = manEmotionClassify(df_male_voices, df_male_target, df_answer)

            elif answer == -1:
                print("Erro na identificação do sexo.")

            # 0: happy
            # 1: fear
            # 2: disgust
            # 3: sad
            # 4: neutral
            # 5: angry
            if emotionAnswer == 0:
                emotionStorage[option][emotionAnswer] = emotionStorage[option][emotionAnswer] + 1

            elif emotionAnswer == 1:
                emotionStorage[option][emotionAnswer] = emotionStorage[option][emotionAnswer] + 1

            elif emotionAnswer == 2:
                emotionStorage[option][emotionAnswer] = emotionStorage[option][emotionAnswer] + 1

            elif emotionAnswer == 3:
                emotionStorage[option][emotionAnswer] = emotionStorage[option][emotionAnswer] + 1

            elif emotionAnswer == 4:
                emotionStorage[option][emotionAnswer] = emotionStorage[option][emotionAnswer] + 1

            elif emotionAnswer == 5:
                emotionStorage[option][emotionAnswer] = emotionStorage[option][emotionAnswer] + 1
            else:
                pass

    elif option == 4:
        print(emotionStorage)

    elif option == 5:
        buildReport(emotionStorage)

    elif option == 6:
        resetValues()

    elif option == 7:
        break